## **A안**
- 종합순위의 활동지수에 장르지수를 추가 합산

In [10]:
import pandas as pd

# Load the Excel file
file_path = "file path"  # Update with your file path
excel_data = pd.ExcelFile(file_path)

# Load each sheet into a DataFrame and clean
def clean_and_standardize(df, sheet_name):
    # Drop first row that contains headers within data
    df.columns = df.iloc[0]
    df = df.drop(0).reset_index(drop=True)
    # Select only relevant columns and rename 'TOTAL 점수' with sheet name for tracking
    df = df[['통일된_이름', 'TOTAL 점수']].copy()
    df.rename(columns={'TOTAL 점수': sheet_name}, inplace=True)
    return df

# Clean and standardize data
dfs = {sheet: clean_and_standardize(excel_data.parse(sheet), sheet) for sheet in excel_data.sheet_names}

# Merge all sheets on '통일된_이름' with outer join to keep all entries
from functools import reduce
combined_df = reduce(lambda left, right: pd.merge(left, right, on='통일된_이름', how='outer'), dfs.values())

# Fill NaN with 0 for accurate summing
combined_df = combined_df.fillna(0)

# Calculate the total score across all sheets
combined_df['TOTAL 점수 합산'] = combined_df[excel_data.sheet_names].sum(axis=1)

# Sort by the total score in descending order
combined_df = combined_df.sort_values(by='TOTAL 점수 합산', ascending=False).reset_index(drop=True)

# Display the resulting DataFrame and export to Excel
combined_df.to_excel("file path", index=False)
combined_df.head(10)  # Display the first 10 rows


C:\Users\TDI\AppData\Local\Temp\ipykernel_1484\2239802748.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df = combined_df.fillna(0)


,통일된_이름,종합순위(top100),K-POP 그룹,발라드,랩힙합,트로트,TOTAL 점수 합산
0,임영웅,225200,20600,92800,6900,445500,791000
1,NewJeans,202900,315300,0,0,0,518200
2,DAY6데이식스,277800,152100,12600,0,0,442500
3,aespa,155900,232600,0,0,0,388500
4,정동원,0,0,2000,0,295200,297200
5,IVE아이브,78700,212900,0,0,0,291600
6,여자아이들,86800,138200,0,2100,0,227100
7,장윤정,0,0,1900,0,218800,220700
8,박지현,0,0,11700,0,197900,209600
9,영탁,0,0,0,0,199700,199700


In [ ]:
import pandas as pd
from functools import reduce

# 엑셀 파일 불러오기
file_path = "file path"
excel_data = pd.ExcelFile(file_path)

# 시트를 데이터프레임으로 불러오고 정리하는 함수
def clean_and_standardize(df, sheet_name):
    df.columns = df.iloc[0]
    df = df.drop(0).reset_index(drop=True)
    df = df[['통일된_이름', 'TOTAL 점수']].copy()
    df.rename(columns={'TOTAL 점수': sheet_name}, inplace=True)
    return df

# 각 시트를 불러와서 표준화
dfs = {sheet: clean_and_standardize(excel_data.parse(sheet), sheet) for sheet in excel_data.sheet_names}

# 첫 번째 시트는 기준 시트로 사용하고 변경 없이 유지
combined_df = dfs[excel_data.sheet_names[0]].copy()

# 가중치 점수를 2번 시트부터 마지막 시트까지 100위까지만 적용
for sheet in excel_data.sheet_names[1:]:
    # 시트에서 상위 100위까지만 가져오기
    limited_df = dfs[sheet].head(100).copy()
    # 각 시트의 실제 행 수에 맞춰 가중치 배열 생성
    weights = [100000 - i * 1000 for i in range(len(limited_df))]
    # 상위 순위에 맞게 가중치 점수를 적용
    limited_df[sheet + '_장르지수'] = weights
    # 기준 시트와 나머지 시트를 '통일된_이름'으로 병합 (outer join)
    combined_df = pd.merge(combined_df, limited_df[['통일된_이름', sheet + '_장르지수']], on='통일된_이름', how='outer')

# NaN을 0으로 채우기
combined_df = combined_df.fillna(0)

# 모든 시트의 가중치 점수를 합산하여 최종 점수 컬럼 생성
weighted_columns = [col for col in combined_df.columns if '_장르지수' in col]
combined_df['활동지수 합계'] = combined_df[weighted_columns].sum(axis=1) + combined_df[excel_data.sheet_names[0]]

# '통일된_이름'이 0인 행을 최종 데이터프레임에서 제거
combined_df = combined_df[combined_df['통일된_이름'] != 0]

# 점수를 내림차순으로 정렬
combined_df = combined_df.sort_values(by='활동지수 합계', ascending=False).reset_index(drop=True)

# 결과를 엑셀 파일로 내보내기
output_path = "file path"
combined_df.to_excel(output_path, index=False)
combined_df.head(10)  # 상위 10개 행 표시


C:\Users\TDI\AppData\Local\Temp\ipykernel_16424\2213191543.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df = combined_df.fillna(0)


,통일된_이름,종합순위(top100),K-POP 그룹_장르지수,발라드_장르지수,랩힙합_장르지수,트로트_장르지수,활동지수 합계
0,임영웅,225200,0.0,99000.0,26000.0,100000.0,450200.0
1,DAY6데이식스,277800,100000.0,43000.0,0.0,0.0,420800.0
2,NewJeans,202900,99000.0,0.0,0.0,0.0,301900.0
3,PLAVE,35900,85000.0,74000.0,87000.0,0.0,281900.0
4,aespa,155900,98000.0,0.0,0.0,0.0,253900.0
5,AKMU악뮤,58200,94000.0,90000.0,0.0,0.0,242200.0
6,세븐틴SEVENTEEN,30000,84000.0,51000.0,76000.0,0.0,241000.0
7,이영지,41900,90000.0,0.0,100000.0,0.0,231900.0
8,QWER,97800,97000.0,21000.0,0.0,0.0,215800.0
9,이클립스ECLIPSE,43500,92000.0,71000.0,8000.0,0.0,214500.0


: 

## **B안**
- 종합순위의 활동지수에 장르지수를 추가 합산한다.
- TOP 100 랭커 + 카테고리별 상위 30위 이내 장르지수

In [11]:
import pandas as pd

# Load the Excel file
file_path = 'file path'  # Update with your file path
excel_data = pd.ExcelFile(file_path)

# Function to clean and standardize data, only keeping top 30 rows for sheets other than the first
def clean_and_standardize(df, sheet_name, is_first_sheet=False):
    # Drop first row that contains headers within data
    df.columns = df.iloc[0]
    df = df.drop(0).reset_index(drop=True)
    # Select only relevant columns and rename 'TOTAL 점수' with sheet name for tracking
    df = df[['통일된_이름', 'TOTAL 점수']].copy()
    df.rename(columns={'TOTAL 점수': sheet_name}, inplace=True)
    # Keep all rows for the first sheet, and top 30 rows for other sheets
    if not is_first_sheet:
        df = df.head(30)
    return df

# Clean and standardize data, applying the top 30 rule to all sheets except the first
sheet_names = excel_data.sheet_names
dfs = {sheet: clean_and_standardize(excel_data.parse(sheet), sheet, is_first_sheet=(i == 0)) for i, sheet in enumerate(sheet_names)}

# Merge all sheets on '통일된_이름' with outer join to keep all entries
from functools import reduce
combined_df2 = reduce(lambda left, right: pd.merge(left, right, on='통일된_이름', how='outer'), dfs.values())

# Fill NaN with 0 for accurate summing
combined_df2 = combined_df2.fillna(0)

# Calculate the total score across all sheets
combined_df2['TOTAL 점수 합산'] = combined_df2[sheet_names].sum(axis=1)

# Sort by the total score in descending order
combined_df2 = combined_df2.sort_values(by='TOTAL 점수 합산', ascending=False).reset_index(drop=True)

# Display the resulting DataFrame and export to Excel
combined_df2.to_excel("file path", index=False)
combined_df2.head(10)  # Display the first 10 rows


C:\Users\TDI\AppData\Local\Temp\ipykernel_1484\4114996550.py:29: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df2 = combined_df2.fillna(0)


,통일된_이름,종합순위(top100),K-POP 그룹,발라드,랩힙합,트로트,TOTAL 점수 합산
0,임영웅,225200,20600,92800,0,445500,784100
1,NewJeans,202900,315300,0,0,0,518200
2,DAY6데이식스,277800,152100,0,0,0,429900
3,aespa,155900,232600,0,0,0,388500
4,정동원,0,0,0,0,295200,295200
5,IVE아이브,78700,212900,0,0,0,291600
6,여자아이들,86800,138200,0,0,0,225000
7,장윤정,0,0,0,0,218800,218800
8,영탁,0,0,0,0,199700,199700
9,박지현,0,0,0,0,197900,197900


In [13]:
import pandas as pd
from functools import reduce

# 엑셀 파일 불러오기
file_path = "file path"
excel_data = pd.ExcelFile(file_path)

# 시트를 데이터프레임으로 불러오고 정리하는 함수
def clean_and_standardize(df, sheet_name):
    df.columns = df.iloc[0]
    df = df.drop(0).reset_index(drop=True)
    df = df[['통일된_이름', 'TOTAL 점수']].copy()
    df.rename(columns={'TOTAL 점수': sheet_name}, inplace=True)
    return df

# 각 시트를 불러와서 표준화
dfs = {sheet: clean_and_standardize(excel_data.parse(sheet), sheet) for sheet in excel_data.sheet_names}

# 첫 번째 시트는 기준 시트로 사용하고 변경 없이 유지
combined_df = dfs[excel_data.sheet_names[0]].copy()

# 각 순위에 대한 점수 리스트 (상위 30위까지)
rank_scores = [
    100000, 95000, 90000, 85000, 80000, 78000, 76000, 74000, 72000, 70000,
    68000, 66000, 64000, 62000, 60000, 58000, 56000, 54000, 52000, 50000,
    48000, 46000, 44000, 42000, 40000, 39000, 38000, 37000, 36000, 35000
]

# 가중치 점수를 2번 시트부터 마지막 시트까지 상위 30위까지만 적용
for sheet in excel_data.sheet_names[1:]:
    # 시트에서 상위 30위까지만 가져오기
    limited_df = dfs[sheet].head(30).copy()
    # 상위 30위에 제공된 점수 리스트를 적용
    limited_df[sheet + '_장르지수'] = rank_scores[:len(limited_df)]
    # 기준 시트와 나머지 시트를 '통일된_이름'으로 병합 (outer join)
    combined_df = pd.merge(combined_df, limited_df[['통일된_이름', sheet + '_장르지수']], on='통일된_이름', how='outer')

# NaN을 0으로 채우기
combined_df = combined_df.fillna(0)

# 모든 시트의 가중치 점수를 합산하여 최종 점수 컬럼 생성
weighted_columns = [col for col in combined_df.columns if '_장르지수' in col]
combined_df['활동지수 합계'] = combined_df[weighted_columns].sum(axis=1) + combined_df[excel_data.sheet_names[0]]

# '통일된_이름'이 0인 행을 최종 데이터프레임에서 제거
combined_df = combined_df[combined_df['통일된_이름'] != 0]

# 점수를 내림차순으로 정렬
combined_df = combined_df.sort_values(by='활동지수 합계', ascending=False).reset_index(drop=True)

# 결과를 엑셀 파일로 내보내기
output_path = "file path"
combined_df.to_excel(output_path, index=False)
combined_df.head(10)  # 상위 10개 행 표시


C:\Users\TDI\AppData\Local\Temp\ipykernel_16424\2253884670.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df = combined_df.fillna(0)


,통일된_이름,종합순위(top100),K-POP 그룹_장르지수,발라드_장르지수,랩힙합_장르지수,트로트_장르지수,활동지수 합계
0,임영웅,225200,0.0,95000.0,0.0,100000.0,420200.0
1,DAY6데이식스,277800,100000.0,0.0,0.0,0.0,377800.0
2,NewJeans,202900,95000.0,0.0,0.0,0.0,297900.0
3,aespa,155900,90000.0,0.0,0.0,0.0,245900.0
4,이영지,41900,68000.0,0.0,100000.0,0.0,209900.0
5,AKMU악뮤,58200,76000.0,68000.0,0.0,0.0,202200.0
6,PLAVE,35900,58000.0,38000.0,62000.0,0.0,193900.0
7,QWER,97800,85000.0,0.0,0.0,0.0,182800.0
8,여자아이들,86800,80000.0,0.0,0.0,0.0,166800.0
9,너드커넥션NerdConnection,38600,62000.0,64000.0,0.0,0.0,164600.0


## **C안**
- 전체 카테고리 합산 데이터를 종합순위화해서 300위까지 노출
- K-pop, 발라드, 랩/힙합, 트로트

In [12]:
import pandas as pd

# Load the Excel file
file_path = 'file path'  # Update with your file path
excel_data = pd.ExcelFile(file_path)

# Function to clean and standardize data
def clean_and_standardize(df, sheet_name):
    # Drop first row that contains headers within data
    df.columns = df.iloc[0]
    df = df.drop(0).reset_index(drop=True)
    # Select only relevant columns and rename 'TOTAL 점수' with sheet name for tracking
    df = df[['통일된_이름', 'TOTAL 점수']].copy()
    df.rename(columns={'TOTAL 점수': sheet_name}, inplace=True)
    return df

# Get sheet names, excluding the first sheet
sheet_names = excel_data.sheet_names
other_sheets = sheet_names[1:]  # Exclude the first sheet

# Load and clean data from all sheets except the first one
dfs = {sheet: clean_and_standardize(excel_data.parse(sheet), sheet) for sheet in other_sheets}

# Merge all non-first sheets on '통일된_이름' with an outer join to keep all entries
from functools import reduce
combined_non_first_df = reduce(lambda left, right: pd.merge(left, right, on='통일된_이름', how='outer'), dfs.values())

# Fill NaN with 0 for accurate summing
combined_non_first_df = combined_non_first_df.fillna(0)

# Calculate the total score across only the non-first sheets
combined_non_first_df['TOTAL 점수 합산'] = combined_non_first_df[other_sheets].sum(axis=1)

# Sort by the total score in descending order
combined_non_first_df = combined_non_first_df.sort_values(by='TOTAL 점수 합산', ascending=False).reset_index(drop=True)

# Display the resulting DataFrame and export to Excel
combined_non_first_df.to_excel("file path", index=False)
combined_non_first_df.head(10)  # Display the first 10 rows


C:\Users\TDI\AppData\Local\Temp\ipykernel_1484\691456136.py:29: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_non_first_df = combined_non_first_df.fillna(0)


,통일된_이름,K-POP 그룹,발라드,랩힙합,트로트,TOTAL 점수 합산
0,임영웅,20600,92800,6900,445500,565800
1,NewJeans,315300,0,0,0,315300
2,정동원,0,2000,0,295200,297200
3,aespa,232600,0,0,0,232600
4,장윤정,0,1900,0,218800,220700
5,IVE아이브,212900,0,0,0,212900
6,박지현,0,11700,0,197900,209600
7,영탁,0,0,0,199700,199700
8,DAY6데이식스,152100,12600,0,0,164700
9,이찬원,0,0,0,148700,148700
